In [65]:
import pandas
import mlflow
import mlflow.sklearn
import optuna
from datetime import datetime
pandas.set_option('display.max_columns', None)

df=pandas.read_json('input_dataset.json')


In [23]:
df

/home/josue/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,has_relevant_water,militiaLine1,spearmanLine1,EaglescoutLine1,has_supplies1,has_squires1,has_arson1,archerLine1,skirmisherLine1,handcanoneerLine1,cavalryarcherLine1,has_thumbring1,has_parthiantactics1,scoutLine1,knightLine1,camelriderLine1,battleelephantLine1,steppelancerLine1,has_bloodlines1,has_husbandry1,batteringramLine1,mangonelLine1,scorpionLine1,bombardcannonLine1,fireshipLine1,demolitionraftLine1,galleyLine1,cannongalleonLine1,has_drydock1,has_shipwright1,watchtowerLine1,stonewallLine1,piercearmorLine1,pierceatkLine1,meleeatkLine1,cavalryarmorLine1,infantryarmorLine1,has_faith1,has_redemption1,has_illumination1,has_atonement1,has_blockprinting1,has_herbalmedicine1,has_theocracy1,has_heresy1,has_sanctity1,has_fervor1,has_masonry1,has_architecture1,has_fortifiedwall1,has_bombardtower1,has_siegeengineers1,has_heatedshot1,has_arrowslits1,has_treadmillcrane1,goldLine1,stoneLine1,woodLine1,foodLine1,has_guilds1,militiaLine2,spearmanLine2,EaglescoutLine2,has_supplies2,has_squires2,has_arson2,archerLine2,skirmisherLine2,handcanoneerLine2,cavalryarcherLine2,has_thumbring2,has_parthiantactics2,scoutLine2,knightLine2,camelriderLine2,battleelephantLine2,steppelancerLine2,has_bloodlines2,has_husbandry2,batteringramLine2,mangonelLine2,scorpionLine2,bombardcannonLine2,fireshipLine2,demolitionraftLine2,galleyLine2,cannongalleonLine2,has_drydock2,has_shipwright2,watchtowerLine2,stonewallLine2,piercearmorLine2,pierceatkLine2,meleeatkLine2,cavalryarmorLine2,infantryarmorLine2,has_faith2,has_redemption2,has_illumination2,has_atonement2,has_blockprinting2,has_herbalmedicine2,has_theocracy2,has_heresy2,has_sanctity2,has_fervor2,has_masonry2,has_architecture2,has_fortifiedwall2,has_bombardtower2,has_siegeengineers2,has_heatedshot2,has_arrowslits2,has_treadmillcrane2,goldLine2,stoneLine2,woodLine2,foodLine2,has_guilds2,duration
0,False,3,3,0,1,0,1,3,2,0,2,1,1,3,3,0,0,0,1,1,2,2,2,0,3,2,3,1,1,1,2,1,3,2,2,3,2,0,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,0,1,2,1,2,2,0,3,3,0,1,1,1,2,2,1,2,0,0,2,3,0,0,0,0,1,2,2,2,1,3,3,3,1,1,0,2,2,2,1,2,3,3,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,2,1,1,2,0,medium
1,False,3,3,0,1,1,1,2,2,0,2,0,0,3,2,0,0,0,1,1,3,3,2,0,3,2,3,1,1,0,2,2,3,1,2,3,3,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,0,1,2,1,2,2,0,2,3,3,0,1,1,3,2,0,0,1,0,0,0,0,0,0,0,0,3,2,2,0,3,3,3,0,1,1,3,2,3,2,2,0,3,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,2,2,2,1,short
2,False,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1,3,3,0,0,1,1,3,2,0,2,1,0,2,2,2,0,0,1,1,3,2,2,0,2,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,2,2,2,1,0,short
3,False,2,3,3,0,1,1,3,2,0,0,1,0,0,0,0,0,0,0,0,3,2,2,0,3,3,3,0,1,1,3,2,3,2,2,0,3,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,2,2,2,1,2,3,0,0,1,1,2,2,0,2,1,1,3,3,0,0,0,1,1,3,1,1,0,2,3,3,0,1,0,1,1,2,2,2,3,2,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,2,1,2,1,1,short
4,False,3,2,0,1,1,1,3,2,1,2,1,1,3,1,2,0,0,1,1,3,3,1,1,2,3,3,2,1,0,3,2,3,2,2,3,3,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,2,1,2,1,0,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68515,False,3,2,0,0,1,1,3,2,0,2,1,1,3,2,2,0,2,1,1,3,3,2,0,3,3,3,1,0,1,2,2,2,2,2,2,3,1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,2,2,1,1,0,3,2,0,1,1,1,2,2,1,2,1,0,3,2,2,0,0,1,1,2,2,2,1,3,3,3,2,1,0,2,2,3,2,2,3,3,1,1,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,2,2,1,2,1,short
68516,False,3,3,0,1,1,1,1,2,1,2,1,0,3,3,0,0,0,1,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,2,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,2,2,1,1,3,3,0,0,1,1,2,2,0,2,1,1,3,3,1,0,2,1,

In [24]:
from sklearn.model_selection import train_test_split

X_matches = df.drop('duration', axis=1)
y_matches = df['duration']
Xtrain, Xtest, ytrain, ytest = train_test_split(X_matches, y_matches, random_state=1)

/home/josue/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
df.head()

,has_relevant_water,militiaLine1,spearmanLine1,EaglescoutLine1,has_supplies1,has_squires1,has_arson1,archerLine1,skirmisherLine1,handcanoneerLine1,cavalryarcherLine1,has_thumbring1,has_parthiantactics1,scoutLine1,knightLine1,camelriderLine1,battleelephantLine1,steppelancerLine1,has_bloodlines1,has_husbandry1,batteringramLine1,mangonelLine1,scorpionLine1,bombardcannonLine1,fireshipLine1,demolitionraftLine1,galleyLine1,cannongalleonLine1,has_drydock1,has_shipwright1,watchtowerLine1,stonewallLine1,piercearmorLine1,pierceatkLine1,meleeatkLine1,cavalryarmorLine1,infantryarmorLine1,has_faith1,has_redemption1,has_illumination1,has_atonement1,has_blockprinting1,has_herbalmedicine1,has_theocracy1,has_heresy1,has_sanctity1,has_fervor1,has_masonry1,has_architecture1,has_fortifiedwall1,has_bombardtower1,has_siegeengineers1,has_heatedshot1,has_arrowslits1,has_treadmillcrane1,goldLine1,stoneLine1,woodLine1,foodLine1,has_guilds1,militiaLine2,spearmanLine2,EaglescoutLine2,has_supplies2,has_squires2,has_arson2,archerLine2,skirmisherLine2,handcanoneerLine2,cavalryarcherLine2,has_thumbring2,has_parthiantactics2,scoutLine2,knightLine2,camelriderLine2,battleelephantLine2,steppelancerLine2,has_bloodlines2,has_husbandry2,batteringramLine2,mangonelLine2,scorpionLine2,bombardcannonLine2,fireshipLine2,demolitionraftLine2,galleyLine2,cannongalleonLine2,has_drydock2,has_shipwright2,watchtowerLine2,stonewallLine2,piercearmorLine2,pierceatkLine2,meleeatkLine2,cavalryarmorLine2,infantryarmorLine2,has_faith2,has_redemption2,has_illumination2,has_atonement2,has_blockprinting2,has_herbalmedicine2,has_theocracy2,has_heresy2,has_sanctity2,has_fervor2,has_masonry2,has_architecture2,has_fortifiedwall2,has_bombardtower2,has_siegeengineers2,has_heatedshot2,has_arrowslits2,has_treadmillcrane2,goldLine2,stoneLine2,woodLine2,foodLine2,has_guilds2,duration
0,False,3,3,0,1,0,1,3,2,0,2,1,1,3,3,0,0,0,1,1,2,2,2,0,3,2,3,1,1,1,2,1,3,2,2,3,2,0,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,0,1,2,1,2,2,0,3,3,0,1,1,1,2,2,1,2,0,0,2,3,0,0,0,0,1,2,2,2,1,3,3,3,1,1,0,2,2,2,1,2,3,3,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,2,1,1,2,0,medium
1,False,3,3,0,1,1,1,2,2,0,2,0,0,3,2,0,0,0,1,1,3,3,2,0,3,2,3,1,1,0,2,2,3,1,2,3,3,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,0,1,2,1,2,2,0,2,3,3,0,1,1,3,2,0,0,1,0,0,0,0,0,0,0,0,3,2,2,0,3,3,3,0,1,1,3,2,3,2,2,0,3,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,2,2,2,1,short
2,False,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1,3,3,0,0,1,1,3,2,0,2,1,0,2,2,2,0,0,1,1,3,2,2,0,2,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,2,2,2,1,0,short
3,False,2,3,3,0,1,1,3,2,0,0,1,0,0,0,0,0,0,0,0,3,2,2,0,3,3,3,0,1,1,3,2,3,2,2,0,3,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,2,2,2,1,2,3,0,0,1,1,2,2,0,2,1,1,3,3,0,0,0,1,1,3,1,1,0,2,3,3,0,1,0,1,1,2,2,2,3,2,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,2,1,2,1,1,short
4,False,3,2,0,1,1,1,3,2,1,2,1,1,3,1,2,0,0,1,1,3,3,1,1,2,3,3,2,1,0,3,2,3,2,2,3,3,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,2,1,2,1,0,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1,short


<h3>Random forest</h3>

In [67]:
from sklearn.ensemble import RandomForestClassifier

RF_best_clf= None
RF_classifier_obj = None

def RF_callback(study, trial):
    global RF_best_clf
    if study.best_trial == trial:
        RF_best_clf = RF_classifier_obj

def RM_objective(trial):
    global RF_classifier_obj
    # Step 2. Setup values for the hyperparameters:
    rf_criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    rf_n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    rf_max_depth = trial.suggest_int("max_depth", 2, 15, log=True)
    
    RF_classifier_obj = RandomForestClassifier(
        max_depth=rf_max_depth,
        n_estimators=rf_n_estimators,
        criterion=rf_criterion
    )

    for step in range(100):
        RF_classifier_obj.fit(Xtrain, ytrain)
        
        # Report intermediate objective value.
        intermediate_value = RF_classifier_obj.score(Xtest, ytest)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()
        return intermediate_value

In [68]:
RF_study = optuna.create_study(direction="maximize")
RF_study.optimize(RM_objective, n_trials=100, callbacks=[RF_callback])

[I 2020-11-16 21:54:33,713] A new study created in memory with name: no-name-26fdcd53-b39e-4454-8078-03bda6fd6cb5
[I 2020-11-16 21:54:35,364] Trial 0 finished with value: 0.7474022183304144 and parameters: {'criterion': 'entropy', 'n_estimators': 47, 'max_depth': 4}. Best is trial 0 with value: 0.7474022183304144.
[I 2020-11-16 21:55:22,609] Trial 1 finished with value: 0.7468768242848803 and parameters: {'criterion': 'entropy', 'n_estimators': 557, 'max_depth': 12}. Best is trial 0 with value: 0.7474022183304144.


In [66]:
now = datetime.now()

timestamp = datetime.timestamp(now)
with mlflow.start_run():
    
    mlflow.log_param("criterion", RF_study.best_trial.params['criterion'])
    mlflow.log_param("n_estimators", RF_study.best_trial.params['n_estimators'])
    mlflow.log_param("max_depth", RF_study.best_trial.params['max_depth'])
    
    mlflow.log_metric("accuracy", RF_study.best_trial.value)
    mlflow.sklearn.log_model(RF_best_clf, "model")

    modelpath = "./mlflow/model-%s-%i" % ("random_forest",timestamp)

    mlflow.sklearn.save_model(RF_best_clf, modelpath)
    
    mlflow.log_artifact("./mlflow/model-%s-%i" % ("random_forest",timestamp))
    
    mlflow.end_run()

print(RF_study.best_trial.value)

0.7474022183304144


/home/josue/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<h3>KNN</h3>

In [73]:
from sklearn.neighbors import KNeighborsClassifier

KNN_best_clf= None
KNN_classifier_obj = None

def KNN_callback(study, trial):
    global KNN_best_clf
    if study.best_trial == trial:
        KNN_best_clf = KNN_classifier_obj

def KNN_objective(trial):
    global KNN_classifier_obj
    # Step 2. Setup values for the hyperparameters:
    knn_weights = trial.suggest_categorical("weights", ["uniform", "distance"])
    knn_p = trial.suggest_int("p", 1, 2)
    knn_n_neighbors = trial.suggest_int("n_neighbors", 2, 10, log=True)
    
    KNN_classifier_obj = KNeighborsClassifier(
        n_neighbors=knn_n_neighbors,
        p=knn_p,
        weights=knn_weights
    )

    for step in range(100):
        KNN_classifier_obj.fit(Xtrain, ytrain)
        
        # Report intermediate objective value.
        intermediate_value = KNN_classifier_obj.score(Xtest, ytest)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()
        return intermediate_value

In [74]:
KNN_study = optuna.create_study(direction="maximize")
KNN_study.optimize(KNN_objective, n_trials=100, callbacks=[KNN_callback])

[I 2020-11-16 22:03:59,547] A new study created in memory with name: no-name-b4177d04-16d8-453f-b3ed-6015092628b2
[I 2020-11-16 22:04:58,672] Trial 0 finished with value: 0.667600700525394 and parameters: {'weights': 'uniform', 'p': 2, 'n_neighbors': 3}. Best is trial 0 with value: 0.667600700525394.
[I 2020-11-16 22:06:03,418] Trial 1 finished with value: 0.6089316987740806 and parameters: {'weights': 'distance', 'p': 1, 'n_neighbors': 4}. Best is trial 0 with value: 0.667600700525394.


In [76]:
now = datetime.now()

timestamp = datetime.timestamp(now)
with mlflow.start_run():
    
    mlflow.log_param("weights", KNN_study.best_trial.params['weights'])
    mlflow.log_param("p", KNN_study.best_trial.params['p'])
    mlflow.log_param("n_neighbors", KNN_study.best_trial.params['n_neighbors'])
    
    mlflow.log_metric("accuracy", KNN_study.best_trial.value)
    mlflow.sklearn.log_model(KNN_best_clf, "model")

    modelpath = "./mlflow/model-%s-%i" % ("KNN",timestamp)

    mlflow.sklearn.save_model(KNN_best_clf, modelpath)
    
    mlflow.log_artifact("./mlflow/model-%s-%i" % ("KNN",timestamp))
    
    mlflow.end_run()

print(KNN_study.best_trial.value)

0.667600700525394


<h3>SVC</h3>

In [84]:
from sklearn.svm import SVC

SVC_best_clf= None
SVC_classifier_obj = None

def SVC_callback(study, trial):
    global SVC_best_clf
    if study.best_trial == trial:
        SVC_best_clf = SVC_classifier_obj

def SVC_objective(trial):
    global SVC_classifier_obj
    # Step 2. Setup values for the hyperparameters:
    svc_C = trial.suggest_categorical("C", [0.1, 1, 10, 100, 1000])
    svc_gamma = trial.suggest_categorical("gamma", [0.1, 1, 10, 100])
    
    SVC_classifier_obj = SVC(
        C=svc_C,
        gamma=svc_gamma
    )

    for step in range(100):
        SVC_classifier_obj.fit(Xtrain, ytrain)
        
        # Report intermediate objective value.
        intermediate_value = SVC_classifier_obj.score(Xtest, ytest)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()
        return intermediate_value

In [85]:
SVC_study = optuna.create_study(direction="maximize")
SVC_study.optimize(SVC_objective, n_trials=100, callbacks=[SVC_callback])

[I 2020-11-16 22:48:39,701] A new study created in memory with name: no-name-4ac06a95-e302-4490-a071-e0259435a0e8
[I 2020-11-16 23:00:18,386] Trial 0 finished with value: 0.7440747227086982 and parameters: {'C': 1000, 'gamma': 1}. Best is trial 0 with value: 0.7440747227086982.
[I 2020-11-16 23:11:35,019] Trial 1 finished with value: 0.7440747227086982 and parameters: {'C': 1, 'gamma': 10}. Best is trial 0 with value: 0.7440747227086982.


In [86]:
now = datetime.now()

timestamp = datetime.timestamp(now)
with mlflow.start_run():
    
    mlflow.log_param("C", SVC_study.best_trial.params['C'])
    mlflow.log_param("gamma", SVC_study.best_trial.params['gamma'])
    
    mlflow.log_metric("accuracy", SVC_study.best_trial.value)
    mlflow.sklearn.log_model(SVC_best_clf, "model")

    modelpath = "./mlflow/model-%s-%i" % ("SVC",timestamp)

    mlflow.sklearn.save_model(SVC_best_clf, modelpath)
    
    mlflow.log_artifact("./mlflow/model-%s-%i" % ("SVC",timestamp))
    
    mlflow.end_run()

print(SVC_study.best_trial.value)

0.7440747227086982


/home/josue/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<h3>Logistic regression</h3>

In [87]:
from sklearn.linear_model import LogisticRegression

LR_best_clf= None
LR_classifier_obj = None

def LR_callback(study, trial):
    global LR_best_clf
    if study.best_trial == trial:
        LR_best_clf = LR_classifier_obj

def LR_objective(trial):
    global LR_classifier_obj
    # Step 2. Setup values for the hyperparameters:
    LR_penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet"])
    LR_solver = trial.suggest_categorical("solver", ["sag", "saga", "lbfgs", "newton-cg"])
    LR_C = trial.suggest_float("C", 0.1, 4, log=True)
    
    LR_classifier_obj = LogisticRegression(
        penalty=LR_penalty,
        C=LR_C,
        solver=LR_solver
    )

    for step in range(100):
        LR_classifier_obj.fit(Xtrain, ytrain)
        
        # Report intermediate objective value.
        intermediate_value = LR_classifier_obj.score(Xtest, ytest)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()
        return intermediate_value

In [88]:
LR_study = optuna.create_study(direction="maximize")
LR_study.optimize(LR_objective, n_trials=100, callbacks=[LR_callback])

[I 2020-11-16 23:36:01,242] A new study created in memory with name: no-name-d017c446-1a95-4c75-9dcc-7f5d54978a83
/home/josue/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2020-11-16 23:36:11,301] Trial 0 finished with value: 0.7474022183304144 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 3.2540531128499777}. Best is trial 0 with value: 0.7474022183304144.
/home/josue/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge


In [89]:
now = datetime.now()

timestamp = datetime.timestamp(now)
with mlflow.start_run():
    
    mlflow.log_param("penalty", LR_study.best_trial.params['penalty'])
    mlflow.log_param("C", LR_study.best_trial.params['C'])
    mlflow.log_param("solver", LR_study.best_trial.params['solver'])
    
    mlflow.log_metric("accuracy", LR_study.best_trial.value)
    mlflow.sklearn.log_model(LR_best_clf, "model")

    modelpath = "./mlflow/model-%s-%i" % ("Logistic regression",timestamp)

    mlflow.sklearn.save_model(LR_best_clf, modelpath)
    
    mlflow.log_artifact("./mlflow/model-%s-%i" % ("Logistic regression",timestamp))
    
    mlflow.end_run()

print(LR_study.best_trial.value)

0.7474022183304144


/home/josue/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
